# Quickstart

In [ ]:
import matplotlib.pyplot as plt

from genesis.version4 import Genesis4

import genesis.version4 as g4

%config InlineBackend.figure_format = 'retina' # Nicer plots

## Load an existing main input file and run Genesis4

Load a pre-existing Genesis 4 main input file - "Example 1: Steady State" from Sven's documentation. The lattice file will be determined automatically from its `&setup` namelist.

In [ ]:
G = Genesis4("data/example1-steadystate/Example1.in")
G.verbose = True

In [ ]:
output = G.run()

LUME-Genesis offers a plotting helpers on the Genesis4 object (and `Genesis4Output` itself) to work with the output data.

You can specify individual data keys to plot the output data and the layout below.

In [ ]:
G.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

## Inspect the output

### Lattice plot

View the lattice data (as interpreted by Genesis4) by interacting with the `output.lattice` object:

In [ ]:
output.lattice.plot();

### Beam and field sizes

In [ ]:
zplot = output.lattice.zplot
field = output.field
plt.plot(zplot, output.beam.xsize * 1e6, label=r"Beam: $\sigma_x$")
plt.plot(zplot, output.beam.ysize * 1e6, label=r"Beam: $\sigma_y$")
plt.plot(zplot, field.xsize * 1e6, label=r"Field: $\sigma_x$")
plt.plot(zplot, field.ysize * 1e6, label=r"Field: $\sigma_y$")
plt.legend()
plt.xlabel(r"$z$ (m)")
plt.ylabel(r"$\sigma_{x,y}$ ($\mu$m)")
plt.ylim([0, 60])
plt.show()

## Make your own input

This section replicates the above Genesis 4-format input entirely in Python.

In [ ]:
main = g4.MainInput(
    [
        g4.Setup(
            rootname="Example1",
            beamline="FEL",
            gamma0=11357.82,
            delz=0.045,
            nbins=8,
            shotnoise=False,
        ),
        g4.LatticeNamelist(zmatch=9.5),
        g4.Field(power=5000.0, waist_size=3e-05, dgrid=0.0002, ngrid=255),
        g4.Beam(delgam=1.0, current=3000.0, ex=4e-07, ey=4e-07),
        g4.Track(),
    ],
)

lattice = g4.Lattice(
    {
        "D1": g4.Drift(L=0.44),
        "D2": g4.Drift(L=0.24),
        "FEL": g4.Line(elements=["FODO"] * 6),
        "FODO": g4.Line(
            elements=["UND", "D1", "QF", "D2", "UND", "D1", "QD", "D2"],
        ),
        "QD": g4.Quadrupole(L=0.08, k1=-2.0),
        "QF": g4.Quadrupole(L=0.08, k1=2.0),
        "UND": g4.Undulator(aw=0.84853, lambdau=0.015, nwig=266, helical=True),
    }
)

In [ ]:
G = Genesis4(main, lattice)
G.verbose = True

In [ ]:
output = G.run()

In [ ]:
G.plot(["beam_xsize", "beam_ysize", "field_xsize", "field_ysize"])

## View available output data

In [ ]:
output.info()

In [ ]:
output["beam_energy"]

## Archive the results to an HDF5 file

In [ ]:
G.archive("quickstart-results.h5")

In [ ]:
restored = Genesis4.from_archive("quickstart-results.h5")

In [ ]:
restored.output.plot()

In [ ]:
G == restored